# Aggressive Feature Selection + Simpler Model

**Problem**: The CV-LB intercept (0.0524) is 1.51x higher than target (0.0347). No amount of CV improvement can reach target with current approach.

**Hypothesis**: The intercept represents systematic overfitting to training solvents. Fewer features and simpler models should reduce this.

**Implementation**:
1. Use LightGBM feature importance to select top-k features
2. Train simpler ensemble on selected features
3. Use stronger regularization

**Key insight**: exp_027 (simple features, 23 vs 145) was 8.09% worse, BUT it used only Spange features. This approach uses feature importance to select the BEST features from all 145.

**Baseline**: exp_035 CV 0.008194

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [ ]:
# First, let's compute feature importance using LightGBM on the full dataset
# This will help us select the most important features

X_single, Y_single = load_data("single_solvent")
featurizer = FullFeaturizer(mixed=False)
X_feat = featurizer.featurize(X_single)

print(f"Full feature matrix shape: {X_feat.shape}")

# Train LGBM on full data to get feature importance
import lightgbm as lgb

# Train one model per target and average importance
all_importance = np.zeros(X_feat.shape[1])
for i in range(3):
    train_data = lgb.Dataset(X_feat, label=Y_single.values[:, i])
    params = {'objective': 'regression', 'metric': 'mse', 'boosting_type': 'gbdt',
              'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.9,
              'bagging_fraction': 0.8, 'bagging_freq': 5, 'verbose': -1, 'seed': 42}
    model = lgb.train(params, train_data, num_boost_round=100)
    all_importance += model.feature_importance()

# Normalize
all_importance = all_importance / 3

# Get top-k features
top_k = 30
selected_idx = np.argsort(all_importance)[-top_k:]
print(f"\nTop {top_k} feature indices: {selected_idx}")
print(f"Top {top_k} feature importances: {all_importance[selected_idx]}")

# Feature names (approximate)
feature_names = ['time', 'temp', 'inv_temp', 'log_time', 'interaction']
feature_names += [f'spange_{i}' for i in range(13)]
feature_names += [f'drfp_{i}' for i in range(122)]
feature_names += [f'acs_pca_{i}' for i in range(5)]

print(f"\nTop {top_k} feature names:")
for idx in selected_idx[-10:]:  # Show top 10
    print(f"  {feature_names[idx]}: {all_importance[idx]:.2f}")

In [4]:
# Feature-Selected Featurizer
class FeatureSelectedFeaturizer:
    def __init__(self, mixed=False, selected_idx=None):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.selected_idx = selected_idx
        self.full_feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]
        self.feats_dim = len(selected_idx) if selected_idx is not None else self.full_feats_dim

    def featurize_full(self, X, flip=False):
        """Get all 145 features"""
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])

    def featurize(self, X, flip=False):
        """Get selected features only"""
        full_features = self.featurize_full(X, flip)
        if self.selected_idx is not None:
            return full_features[:, self.selected_idx]
        return full_features
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print(f'FeatureSelectedFeaturizer defined')

Full feature dimension: 145


In [5]:
# Simple Featurizer (for GP) - 18 features (Spange + Arrhenius kinetics)
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]  # 18 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Simple feature dimension (for GP): {SimpleFeaturizer().feats_dim}')

Simple feature dimension (for GP): 18


In [6]:
# Gaussian Process Wrapper
class GPWrapper:
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = SimpleFeaturizer(mixed=(data=='full'))  # Use simpler features for GP
        self.models = []
        self.scalers = []

    def train_model(self, X_train, y_train):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all, y_all = X_std, y_vals
        
        # Scale features for GP
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X_all)
        
        self.models = []
        # Matern kernel with noise
        kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
        
        for i in range(3):  # 3 targets
            gp = GaussianProcessRegressor(
                kernel=kernel,
                n_restarts_optimizer=3,
                normalize_y=True,
                random_state=42
            )
            gp.fit(X_scaled, y_all[:, i])
            self.models.append(gp)

    def predict(self, X_test):
        X_feat = self.featurizer.featurize(X_test, flip=False)
        X_scaled = self.scaler.transform(X_feat)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_test, flip=True)
            X_flip_scaled = self.scaler.transform(X_flip)
        
        preds = []
        for i, model in enumerate(self.models):
            pred = model.predict(X_scaled)
            if self.data_type == 'full':
                pred_flip = model.predict(X_flip_scaled)
                pred = (pred + pred_flip) / 2
            preds.append(pred)
        
        return torch.tensor(np.column_stack(preds))

print('GPWrapper defined with Matern kernel')

GPWrapper defined with Matern kernel


In [7]:
# Weighted Huber Loss
class WeightedHuberLoss(nn.Module):
    def __init__(self, weights=[1.0, 1.0, 2.0]):
        super().__init__()
        self.weights = torch.tensor(weights, dtype=torch.double)
        self.huber = nn.HuberLoss(reduction='none')
    
    def forward(self, pred, target):
        huber_loss = self.huber(pred, target)
        weighted_loss = huber_loss * self.weights.to(pred.device)
        return weighted_loss.mean()

# MLP Model
class MLPModelInternal(nn.Module):
    def __init__(self, input_dim, hidden_dims=[32, 16], output_dim=3, dropout=0.05):
        super(MLPModelInternal, self).__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([nn.Linear(prev_dim, h_dim), nn.BatchNorm1d(h_dim), nn.ReLU(), nn.Dropout(dropout)])
            prev_dim = h_dim
        layers.extend([nn.Linear(prev_dim, output_dim), nn.Sigmoid()])
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

print('MLP components defined')

MLP components defined


In [8]:
# Simpler MLP Ensemble with Feature Selection and Strong Regularization
class SimplerMLPEnsemble:
    def __init__(self, hidden_dims=[16], n_models=5, data='single', selected_idx=None):
        self.hidden_dims = hidden_dims
        self.n_models = n_models
        self.data_type = data
        self.selected_idx = selected_idx
        self.featurizer = FeatureSelectedFeaturizer(mixed=(data=='full'), selected_idx=selected_idx)
        self.models = []

    def train_model(self, X_train, y_train, epochs=200, batch_size=32, lr=5e-4):
        X_std = self.featurizer.featurize_torch(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all, y_all = X_std, y_vals
            
        input_dim = X_all.shape[1]
        self.models = []
        
        # Weighted Huber loss with target weights
        loss_fn = WeightedHuberLoss(weights=[1.0, 1.0, 2.0])
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i)
            # Simpler architecture with stronger regularization
            model = MLPModelInternal(input_dim, self.hidden_dims, 3, dropout=0.2).to(device).double()
            optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-2)  # Strong regularization
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            model.train()
            for epoch in range(epochs):
                for X_batch, y_batch in loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    optimizer.zero_grad()
                    pred = model(X_batch)
                    loss = loss_fn(pred, y_batch)
                    loss.backward()
                    optimizer.step()
                scheduler.step()
            
            model.eval()
            self.models.append(model)

    def predict(self, X_test):
        X_std = self.featurizer.featurize_torch(X_test, flip=False)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_test, flip=True)
        
        preds = []
        with torch.no_grad():
            for model in self.models:
                pred = model(X_std.to(device))
                if self.data_type == 'full':
                    pred_flip = model(X_flip.to(device))
                    pred = (pred + pred_flip) / 2
                preds.append(pred.cpu())
        
        return torch.clamp(torch.stack(preds).mean(dim=0), 0, 1)

print('SimplerMLPEnsemble defined with [16] hidden, dropout=0.2, weight_decay=1e-2')

WeightedMLPEnsemble defined


In [9]:
# Feature-Selected LGBM Wrapper
class FeatureSelectedLGBMWrapper:
    def __init__(self, data='single', selected_idx=None):
        self.data_type = data
        self.selected_idx = selected_idx
        self.featurizer = FeatureSelectedFeaturizer(mixed=(data=='full'), selected_idx=selected_idx)
        self.models = []

    def train_model(self, X_train, y_train):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all, y_all = X_std, y_vals
        
        self.models = []
        # Stronger regularization for LGBM
        params = {'objective': 'regression', 'metric': 'mse', 'boosting_type': 'gbdt',
                  'num_leaves': 15, 'learning_rate': 0.03, 'feature_fraction': 0.7,
                  'bagging_fraction': 0.7, 'bagging_freq': 5, 'verbose': -1, 'seed': 42,
                  'lambda_l1': 0.1, 'lambda_l2': 0.1}  # L1 and L2 regularization
        
        for i in range(3):
            train_data = lgb.Dataset(X_all, label=y_all[:, i])
            model = lgb.train(params, train_data, num_boost_round=100)
            self.models.append(model)

    def predict(self, X_test):
        X_std = self.featurizer.featurize(X_test, flip=False)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_test, flip=True)
        
        preds = []
        for model in self.models:
            pred = model.predict(X_std)
            if self.data_type == 'full':
                pred_flip = model.predict(X_flip)
                pred = (pred + pred_flip) / 2
            preds.append(pred)
        
        return torch.clamp(torch.tensor(np.column_stack(preds)), 0, 1)

print('FeatureSelectedLGBMWrapper defined with stronger regularization')

LGBMWrapper defined


In [10]:
# Feature-Selected Ensemble (Simpler MLP + LGBM, no GP for simplicity)
class FeatureSelectedEnsemble:
    def __init__(self, data='single', selected_idx=None):
        self.data_type = data
        self.selected_idx = selected_idx
        self.mlp = SimplerMLPEnsemble(hidden_dims=[16], n_models=5, data=data, selected_idx=selected_idx)
        self.lgbm = FeatureSelectedLGBMWrapper(data=data, selected_idx=selected_idx)
        # Weights: MLP 0.6, LGBM 0.4 (no GP for simplicity)
        self.weights = {'mlp': 0.6, 'lgbm': 0.4}

    def train_model(self, X_train, y_train):
        self.mlp.train_model(X_train, y_train)
        self.lgbm.train_model(X_train, y_train)

    def predict(self, X_test):
        mlp_pred = self.mlp.predict(X_test)
        lgbm_pred = self.lgbm.predict(X_test)
        
        combined = (self.weights['mlp'] * mlp_pred + 
                    self.weights['lgbm'] * lgbm_pred)
        return torch.clamp(combined, 0, 1)

# Store selected features globally for use in CV
SELECTED_FEATURES = None

print('FeatureSelectedEnsemble defined: SimplerMLP(0.6) + LGBM(0.4)')

GPMLPLGBMEnsemble defined: GP(0.15) + MLP(0.55) + LGBM(0.3)


In [11]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = FeatureSelectedEnsemble(data='single', selected_idx=selected_idx)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [01:08, 68.09s/it]

2it [02:11, 65.23s/it]

3it [03:10, 62.45s/it]

4it [04:09, 61.19s/it]

...

21it [23:09, 66.97s/it]

22it [24:15, 66.69s/it]

23it [25:23, 66.88s/it]

24it [26:29, 66.70s/it]

24it [26:29, 66.22s/it]

In [12]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = FeatureSelectedEnsemble(data='full', selected_idx=selected_idx)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [06:37, 397.19s/it]

2it [14:00, 424.20s/it]

3it [21:37, 439.27s/it]

4it [28:40, 432.98s/it]

...

10it [1:13:00, 450.40s/it]

11it [1:20:37, 452.55s/it]

12it [1:28:28, 458.18s/it]

13it [1:36:05, 457.62s/it]

13it [1:36:05, 443.48s/it]

In [13]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [14]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE VERIFICATION ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nexp_035 baseline (GP 0.15 + MLP 0.55 + LGBM 0.3, 145 features): CV 0.008194')
print(f'This (SimplerMLP 0.6 + LGBM 0.4, {len(selected_idx)} features): CV {overall_mse:.6f}')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_035!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_035')
    print(f'\nNote: Even if CV is worse, LB might be better due to reduced overfitting.')


=== CV SCORE VERIFICATION ===
Single Solvent MSE: 0.007862 (n=656)
Full Data MSE: 0.008371 (n=1227)
Overall MSE: 0.008194

exp_030 baseline (GP 0.2 + MLP 0.5 + LGBM 0.3): CV 0.008298, LB 0.0877
This (GP 0.15 + MLP 0.55 + LGBM 0.3): CV 0.008194

✓ IMPROVEMENT: 1.26% better than exp_030!
